In [2]:
import pandas as pd
import numpy as np

In [3]:
import json

with open('../../nodes.json', 'r', encoding='utf-8') as f:
    json_file = json.load(f)

nodes = pd.json_normalize(json_file['people'])

In [4]:
nodes = nodes.dropna(subset=['first_name'])

In [5]:
nodes = nodes.drop_duplicates(subset=['id'])

In [6]:
nodes

,id,first_name,last_name,domain,photo_file,photo_url,meta.city.id,meta.city.title,meta.can_see_audio,meta.interests,...,meta.status_audio.owner_id,meta.status_audio.title,meta.status_audio.duration,meta.status_audio.is_explicit,meta.status_audio.is_focus_track,meta.status_audio.track_code,meta.status_audio.url,meta.status_audio.stream_duration,meta.status_audio.date,meta.status_audio.genre_id
0,179301939,Lyubov,Novoselova,id179301939,None,https://sun1-95.userapi.com/impg/DW4IDqvukChyc...,501.0,Sarapul,1.0,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,327327093,Anzhela,Levkina,id327327093,None,https://sun1-16.userapi.com/s/v1/ig1/VNiVPfPXI...,281.0,Brest,1.0,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,288357777,Vasily,Volkov,id288357777,None,https://sun1-24.userapi.com/s/v1/ig1/y6Dnjgjvq...,1032.0,Polotsk,1.0,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,55846307,Pashka,Polyakov,id55846307,None,https://sun1-18.userapi.com/s/v1/ig1/xdvoQkERj...,1463.0,Zelenograd,1.0,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,210288,Yana,Vasilyeva,dabasssista,None,https://sun1-23.userapi.com/s/v1/ig2/Z4emj1vbY...,1.0,Moscow,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8346,59192221,Gulshat,Ilshatovna,id59192221,None,https://sun1-20.userapi.com/s/v1/ig2/5X_P4-WTP...,NaN,NaN,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8347,59259047,Lera,Shukaylo,inkognito___vl,None,https://sun1-56.userapi.com/s/v1/ig2/7ZzD51eFE...,88.0,Naberezhnye Chelny,1.0,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8348,59285071,Gulia,Nizamutdinova,id59285071,None,https://sun1-26.userapi.com/s/v1/ig2/Voh4hHi_y...,88.0,Naberezhnye Chelny,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8349,60031769,Vlad,Mitenkov,id60031769,None,https://sun1-97.userapi.com/impg/_nb2NhfE_jv_q...,49.0,Yekaterinburg,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
edges = pd.read_csv('../../edges.csv')

In [8]:
edges

,target_user_id,interactor_id,likes_count,comments_count,reposts_count
0,260762184,189326838,1,0,0
1,260762184,260762184,1,0,0
2,260762184,370224313,1,0,0
3,260762184,400282413,1,0,0
4,260762184,367322161,1,0,0
...,...,...,...,...,...
86134,224120056,814004012,1,0,0
86135,224120056,821004817,1,0,0
86136,224120056,120942437,1,0,0
86137,224120056,482835606,1,0,0


In [30]:
with open('../../nodes.json', 'r', encoding='utf-8') as f:
    json_file = json.load(f)

friendship = pd.DataFrame(json_file['edges']).drop(columns=[2]).rename(columns={0: 'user_id', 1: 'friend_id'})

In [31]:
friendship = friendship.dropna()
friendship = friendship.drop_duplicates(subset=['user_id', 'friend_id'])

In [32]:
friendship

,user_id,friend_id
0,288357777,55846307
2,55846307,210288
3,55846307,278285
4,55846307,444271
5,55846307,586269
...,...,...
19488,189244363,20232183
19489,189244363,34613926
19490,189244363,35951655
19491,189244363,30414089


In [33]:
friendship_set = set(friendship.apply(lambda row: frozenset([row['user_id'], row['friend_id']]), axis=1))


def is_friend_pair(row):
    pair = frozenset([row['target_user_id'], row['interactor_id']])
    return pair in friendship_set


edges['is_friend'] = edges.apply(is_friend_pair, axis=1)

In [35]:
edges['is_friend'].value_counts()

is_friend
False    85884
True       255
Name: count, dtype: int64

In [37]:
nodes[nodes['id'] == 288357777]

,id,first_name,last_name,domain,photo_file,photo_url,meta.city.id,meta.city.title,meta.can_see_audio,meta.interests,...,meta.status_audio.owner_id,meta.status_audio.title,meta.status_audio.duration,meta.status_audio.is_explicit,meta.status_audio.is_focus_track,meta.status_audio.track_code,meta.status_audio.url,meta.status_audio.stream_duration,meta.status_audio.date,meta.status_audio.genre_id
4,288357777,Vasily,Volkov,id288357777,None,https://sun1-24.userapi.com/s/v1/ig1/y6Dnjgjvq...,1032.0,Polotsk,1.0,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [45]:
edges.iloc[100]

target_user_id    283301972
interactor_id     336410992
likes_count               1
comments_count            0
reposts_count             0
is_friend             False
Name: 100, dtype: object

In [94]:
edges = edges.drop(columns=['reposts_count'])

In [95]:
edges[edges['is_friend']==True]

,target_user_id,interactor_id,likes_count,comments_count,is_friend
129,55846307,1590824,1,1,True
130,55846307,2559871,4,0,True
131,55846307,5123553,1,0,True
133,55846307,6432619,1,0,True
134,55846307,8220590,1,0,True
...,...,...,...,...,...
55180,432805423,202629375,1,0,True
55181,432805423,281302757,1,0,True
55183,432805423,378701994,1,0,True
72920,28978854,432805423,2,0,True


In [160]:
edges[edges['target_user_id']==281302757]

,target_user_id,interactor_id,likes_count,comments_count,is_friend


In [124]:
s = len(set(edges['interactor_id'].unique()) & set(edges['target_user_id']))
print(s)

771


In [156]:
edges['comments_count'].value_counts()

comments_count
0      83292
1       2265
2        422
3         76
4         45
5         13
6          8
7          3
12         2
8          2
9          2
36         1
33         1
77         1
328        1
17         1
13         1
64         1
25         1
498        1
Name: count, dtype: int64

In [153]:
nodes[nodes['id'] == 482835606]

,id,first_name,last_name,domain,photo_file,photo_url,meta.city.id,meta.city.title,meta.can_see_audio,meta.interests,...,meta.status_audio.owner_id,meta.status_audio.title,meta.status_audio.duration,meta.status_audio.is_explicit,meta.status_audio.is_focus_track,meta.status_audio.track_code,meta.status_audio.url,meta.status_audio.stream_duration,meta.status_audio.date,meta.status_audio.genre_id


In [149]:
friendship[friendship['user_id']==260762184]

,user_id,friend_id


In [ ]:
for i in edges:
    if not (friendship.loc[i]).any():
    print('+')

+


In [134]:
len(edges['interactor_id'].unique())

77462

In [126]:
nodes['id']

0       179301939
2       327327093
4       288357777
5        55846307
8          210288
          ...    
8346     59192221
8347     59259047
8348     59285071
8349     60031769
8350     60137743
Name: id, Length: 6714, dtype: int64

In [58]:
have_friends = friendship['user_id']

In [69]:
shf = set(have_friends)

In [70]:
len(shf)

120

In [ ]:
len(edges['interactor_id'].value_counts())

77462

In [165]:
set(edges['interactor_id']) & set(edges['target_user_id'])

{1186205,
 1939920,
 2096458,
 2529463,
 2888163,
 2992385,
 3336589,
 3654241,
 3822632,
 3993225,
 4183711,
 4777171,
 4796540,
 5123553,
 5267669,
 6432619,
 6505209,
 7571350,
 8030497,
 8045670,
 8292833,
 8853225,
 9068194,
 9923336,
 10059255,
 10085279,
 10428907,
 11728726,
 12038899,
 12756666,
 13976479,
 14040748,
 14499087,
 14634020,
 17192893,
 19440457,
 19694780,
 20214034,
 20699390,
 20708651,
 22330354,
 22935063,
 24014538,
 24980376,
 26520039,
 26661366,
 27110902,
 27295701,
 28978854,
 29341724,
 29594861,
 30028655,
 30078265,
 31781662,
 34490689,
 34675402,
 34777600,
 35729654,
 36572399,
 37176576,
 38402115,
 39371659,
 41616104,
 42118770,
 47094883,
 49531240,
 49672579,
 50486094,
 50706057,
 51278027,
 51952587,
 53809248,
 54327597,
 54572101,
 54918623,
 54971617,
 55846307,
 59997518,
 60015021,
 60220605,
 61325544,
 61458119,
 61828309,
 62043573,
 64561543,
 66281180,
 66353461,
 66742299,
 68816480,
 70152728,
 72908141,
 73055924,
 74290648,
 

In [163]:
len(edges['target_user_id'].value_counts())

1052

In [166]:
sampled_edges = edges.drop_duplicates(subset='target_user_id')

In [173]:
set_ids = set(sampled_edges['target_user_id']) | set(sampled_edges['interactor_id'])

In [176]:
for i in set_ids:
    print(i)

2441217
701898754
569266179
468766724
4554762
189575179
9576459
286253072
143892497
230125586
19279894
151539737
382922778
661975067
147230747
1040261152
235810849
197283874
77467684
185364517
235532328
342929449
131842089
431837227
719765547
710262833
450805810
295247923
14225460
86937649
372076598
775565368
58150974
382337087
283869248
444276804
24644
64184389
140013638
134312012
35102797
173125714
706130002
1355861
156659803
622063710
5115999
53809248
707702881
684499042
14778464
858792042
246329452
163749998
468795503
323866736
4575344
741535858
6500468
402628725
396890230
378040439
262328440
477962361
257122427
4796540
582602877
766763133
138014846
271847553
158748809
836477068
116818063
422781076
408305812
205709461
243757212
134168736
31891624
98668712
627876013
562032814
489427123
470212788
74354869
718540985
56910010
189923515
135106748
136581308
745509054
299811007
394326210
241475785
51278027
102727885
746057933
177303761
188149972
627577045
5267669
151273687
225501399
27252

In [66]:
nodes_without_friends = nodes[~nodes['id'].isin(shf)]

In [76]:
nodes_without_friends.to_csv('1.csv')

In [84]:
friendship[friendship['user_id'].isin(friendship['friend_id'])]['user_id'].unique()

array([288357777,  55846307, 592630078, 332323773, 425554768, 496597108,
       360981711, 382089357, 355512739, 371404462, 754110083, 460132198,
       432805423, 141263294, 137953186, 272332784, 324794954, 454723886,
       849853976, 395926934, 404284509, 464651141, 375431034, 231957672,
       371235977,  32139041,  13212791, 345684715, 335012059, 586593132,
       583602971, 389763287, 375532131,  54966692, 394460318, 395474310,
       197833289, 582564854, 723206013, 848843702, 380221737, 374539357,
       746055142, 593319553, 286603903, 159700076,  96903539,  41191971,
       177195982, 164560631, 173818302, 152247182,  20371699, 153872531,
       136493605, 122570289, 137723804, 237805392, 463419391, 420567656,
       218946760, 170307138, 454321589, 343063348, 744967389, 782036426,
       382025708, 541164978, 539436914,  28958461,  28782936, 431838326,
       431524100, 317661612, 314814303, 815348202, 188697522, 455559973,
         1890598])